# 加速程序运行

python代码相比于C++、Java等运行还是慢的，虽然我们大多数时候感觉不明显，但是还是要注意尽可能写出高效率的代码。下面看一点简单的注意事项。

## 给程序性能测试

测试程序运行所花费的时间并做性能测试。如果只是简单的想测试下程序整体花费的时间， 通常使用Unix时间函数就行了，比如：

```code
bash % time python3 someprogram.py
real 0m13.937s
user 0m12.162s
sys  0m0.098s
bash %
```

如果你还需要一个程序各个细节的详细报告，可以使用 cProfile 模块：

```code
bash % python3 -m cProfile someprogram.py
bash %
```

不过通常情况是介于这两个极端之间。比如已经知道代码运行时在少数几个函数中花费了绝大部分时间。 对于这些函数的性能测试，可以使用一个简单的装饰器。要使用这个装饰器，只需要将其放置在要进行性能测试的函数定义前即可。

In [138]:
# timethis.py

import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

@timethis
def countdown(n):
     while n > 0:
            n -= 1

countdown(10000000)

__main__.countdown : 1.0297941000000037


对于测试很小的代码片段运行性能，使用 timeit 模块会很方便

In [139]:
from timeit import timeit
timeit('math.sqrt(2)', 'import math')
timeit('sqrt(2)', 'from math import sqrt')
timeit('math.sqrt(2)', 'import math', number=10000000)
timeit('sqrt(2)', 'from math import sqrt', number=10000000)

1.3171404999999936

当执行性能测试的时候，需要注意的是你获取的结果都是近似值。 time.perf_counter() 函数会在给定平台上获取最高精度的计时值。 不过，它仍然还是基于时钟时间，很多因素会影响到它的精确度，比如机器负载。 如果你对于执行时间更感兴趣，使用 time.process_time() 来代替它。

In [140]:
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

## 如何加速程序运行

程序运行太慢，想在不使用复杂技术比如C扩展或JIT编译器的情况下加快程序运行速度。

关于程序优化的第一个准则是“不要优化”，第二个准则是“不要优化那些无关紧要的部分”。 如果你的程序运行缓慢，首先得对它进行性能测试找到问题所在。

通常来讲会发现程序在少数几个热点地方花费了大量时间， 比如内存的数据处理循环。一旦定位到这些点，就可以使用下面这些实用技术来加速程序运行。

### 使用函数

很多程序员刚开始会使用Python语言写一些简单脚本。 当编写脚本的时候，通常习惯了写毫无结构的代码。比如：

```python
# somescript.py

import sys
import csv

with open(sys.argv[1]) as f:
     for row in csv.reader(f):

         # Some kind of processing
         pass
```

像这样定义在全局范围的代码运行起来要比定义在函数中运行慢的多。 这种速度差异是由于局部变量和全局变量的实现方式（**使用局部变量要更快些**）。 因此，如果想让程序运行更快些，只需要将脚本语句放入函数中即可：

```python
# somescript.py
import sys
import csv

def main(filename):
    with open(filename) as f:
         for row in csv.reader(f):
             # Some kind of processing
             pass

main(sys.argv[1])
```

根据经验，使用函数带来15-30%的性能提升是很常见的。

局部变量会比全局变量运行速度快。 对于频繁访问的名称，通过将这些名称变成局部变量可以加速程序运行。

对于类中的属性访问也同样适用于这个原理。 通常来讲，查找某个值比如 self.name 会比访问一个局部变量要慢一些。 在内部循环中，可以将某个需要频繁访问的属性放入到一个局部变量中。

### 尽可能去掉属性访问

每一次**使用点(.)操作符来访问属性的时候会带来额外的开销**。 它会触发特定的方法，比如 __getattribute__() 和 __getattr__() ，这些方法会进行字典操作操作。

通常你可以使用 from module import name 这样的导入形式，以及使用绑定的方法。比如下面的函数是耗时的：

```python
import math

def compute_roots(nums):
    result = []
    for n in nums:
        result.append(math.sqrt(n))
    return result

# Test
nums = range(1000000)
for n in range(100):
    r = compute_roots(nums)
```

可以修改compute_roots函数如下：

In [141]:
from math import sqrt

def compute_roots(nums):

    result = []
    result_append = result.append
    for n in nums:
        result_append(sqrt(n))
    return result

修改后的版本运行时间会减少一些。唯一不同之处就是消除了属性访问。 用 sqrt() 代替了 math.sqrt() 。 The result.append() 方法被赋给一个局部变量 result_append ，然后在内部循环中使用它。

这些改变只有在大量重复代码中才有意义，比如循环。 因此，这些优化也只是在某些特定地方才应该被使用。

### 避免不必要的抽象

任何时候当你使用额外的处理层（比如装饰器、属性访问、描述器）去包装你的代码时，都会让程序运行变慢。

In [142]:
class A:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    @property
    def y(self):
        return self._y
    @y.setter
    def y(self, value):
        self._y = value
        
from timeit import timeit
a = A(1,2)
timeit('a.x', 'from __main__ import a')

0.044568299999980354

In [143]:
timeit('a.y', 'from __main__ import a')

0.14513039999999933

访问属性y相比属性x而言慢的不止一点点，大概慢了4.5倍。 如果你在意性能的话，那么就需要重新审视下对于y的属性访问器的定义是否真的有必要了。 如果没有必要，就使用简单属性吧。 如果仅仅是因为其他编程语言需要使用getter/setter函数就去修改代码风格，这个真的没有必要。

### 使用内置的容器

内置的数据类型比如字符串、元组、列表、集合和字典都是使用C来实现的，运行起来非常快。 如果想自己实现新的数据结构（比如链接列表、平衡树等）， 那么要想在性能上达到内置的速度几乎不可能，因此，还是乖乖的使用内置的吧。

另外，还要避免创建不必要的数据结构或复制。

### 并行编程

这部分有参考：[Python性能优化的20条建议](https://segmentfault.com/a/1190000000666603)。

可以通过内置的模块multiprocessing实现下面几种并行模式：

多进程：对于CPU密集型的程序，可以使用multiprocessing的Process,Pool等封装好的类，通过多进程的方式实现并行计算。但是因为进程中的通信成本比较大，对于进程之间需要大量数据交互的程序效率未必有大的提高。

多线程：对于IO密集型的程序，multiprocessing.dummy模块使用multiprocessing的接口封装threading，使得多线程编程也变得非常轻松(比如可以使用Pool的map接口，简洁高效)。

分布式：multiprocessing中的Managers类提供了可以在不同进程之共享数据的方式，可以在此基础上开发出分布式的程序。

不同的业务场景可以选择其中的一种或几种的组合实现程序性能的优化。

下一节稍微介绍一点并行编程的概念。

### 讨论

**在优化之前，有必要先研究下使用的算法**。 选择一个复杂度为 O(n log n) 的算法要比你去调整一个复杂度为 O(n**2) 的算法所带来的性能提升要大得多。

如果你觉得你还是得进行优化，那么请从整体考虑。 作为一般准则，不要对程序的每一个部分都去优化,因为这些修改会导致代码难以阅读和理解。 你应该**专注于优化产生性能瓶颈的地方，比如内部循环**。

对循环的优化所遵循的原则是尽量减少循环过程中的计算量，有多重循环的尽量将内层的计算提到上一层———[Python 代码性能优化技巧](https://www.ibm.com/developerworks/cn/linux/l-cn-python-optim/index.html)。

这里对循环做些补充，参考：[Python性能诀窍](http://pfmiles.github.io/blog/python-speed-performance-tips/).

Python支持好几种循环结构。for语句是最常用的。它遍历一个序列的每个元素，将每个元素赋值给循环变量。如果你的循环体很简单，for循环本身的解释成本将占据大部分的开销。这个时候**map函数**就能派上用场了。你可以将map函数看作是for循环采用C代码来实现。唯一的约束是“**循环体”必须是一个函数调用**。**list comprehension 列表生成式**除了语法上的便利性之外，他们常常和等价的map调用一样快甚至更快。比如：

```python
newlist = []
for word in oldlist:
    newlist.append(word.upper())
```

可以使用map函数将这个循环由解释执行推到编译好的C代码中去执行：

```python
newlist = map(str.upper, oldlist)
```

List comprehension在python 2.0的时候被加入。它们提供了一种更紧凑的语法和更高效的方式来表达上面的for循环：

```python
newlist = [s.upper() for s in oldlist]
```

如果优化要求比较高，本节的这些简单技术满足不了，那么可以研究下基于即时编译（JIT）技术的一些工具。 例如，PyPy工程是Python解释器的另外一种实现，它会分析程序运行并对那些频繁执行的部分生成本机机器码。 它有时候能极大的提升性能，通常可以接近C代码的速度。 不过可惜的是，PyPy还不能完全支持Python3.。

还可以考虑下Numba工程， Numba是一个在你使用装饰器来选择Python函数进行优化时的动态编译器。 这些函数会使用LLVM被编译成本地机器码。它同样可以极大的提升性能。 但是，跟PyPy一样，它对于Python 3的支持现在还停留在实验阶段。

最后引用John Ousterhout说过的话作为结尾：“最好的性能优化是从不工作到工作状态的迁移”。 直到你真的需要优化的时候再去考虑它。确保你程序正确的运行通常比让它运行更快要更重要一些（至少开始是这样的）.

## 并行编程

对于并行编程, Python有多种长期支持的方法, 包括**多线程**, **调用子进程**, 以及各种各样的关于**生成器函数**的技巧。这部分将会简单记录并发编程各种方面的技巧, 包括通用的**多线程技术**以及**并行计算的实现方法**。并发的程序有潜在的危险. 因此, 要注意能给出更加可信赖和易调试的代码。这里只是简单介绍，更多关于并行计算的内容会在7-parallel-programming文件夹中记录。

### 启动与停止线程

为需要并发执行的代码创建/销毁线程：**threading 库**可以**在单独的线程中执行任何的在 Python 中可以调用的对象**。可以创建一个 Thread 对象并将你要执行的对象以 target 参数的形式提供给该对象。

In [49]:
import time
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(1)

# Create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(3,))
t.start()

T-minus 3


当创建好一个线程对象后，该对象**并不会立即执行**，除非你调用它的 **start() 方法**（当你调用 start() 方法时，它会调用你传递进来的函数，并把你传递进来的参数传递给该函数）。Python中的线程会在一个单独的系统级线程中执行（比如说一个 POSIX 线程或者一个 Windows 线程），这些线程将由操作系统来全权管理。线程一旦启动，将独立执行直到目标函数返回。

可以查询一个线程对象的状态，看它是否还在执行：

In [50]:
if t.is_alive():
    print('Still running')
else:
    print('Completed')

Still running
T-minus 2


Python解释器直到所有线程都终止前仍保持运行。对于需要长时间运行的线程或者需要一直运行的后台任务，你应当考虑使用后台线程。
```python
# 使用daemon
t = Thread(target=countdown, args=(10,), daemon=True)
t.start()
```

由于全局解释锁（GIL）的原因，Python 的线程被限制到同一时刻只允许一个线程执行这样一个执行模型。所以，Python 的线程更适用于处理I/O和其他需要并发执行的阻塞操作（比如等待I/O、等待从数据库获取数据等等），而**不是需要多处理器并行的计算密集型任务**。所以这块暂时不需要太多关注。

### 简单并行编程

有个程序要执行CPU密集型工作，想让他利用**多核CPU**的优势来运行的快一点。

concurrent.futures 库提供了一个 ProcessPoolExecutor 类， 可被用来在一个单独的Python解释器中执行计算密集型函数。 不过，要使用它，首先要有一些计算密集型的任务。一个脚本，在这些日志文件中查找出所有访问过robots.txt文件的主机(因为我没有日志文件，所以没运行结果)

In [51]:
# findrobots.py

import gzip
import io
import glob

def find_robots(filename):
    '''
    Find all of the hosts that access robots.txt in a single log file
    '''
    robots = set()
    with gzip.open(filename) as f:
        for line in io.TextIOWrapper(f,encoding='ascii'):
            fields = line.split()
            if fields[6] == '/robots.txt':
                robots.add(fields[0])
    return robots

def find_all_robots(logdir):
    '''
    Find all hosts across and entire sequence of files
    '''
    files = glob.glob(logdir+'/*.log.gz')
    all_robots = set()
    for robots in map(find_robots, files):
        all_robots.update(robots)
    return all_robots

if __name__ == '__main__':
    robots = find_all_robots('logs')
    for ipaddr in robots:
        print(ipaddr)

T-minus 1


前面的程序使用了通常的**map-reduce风格**来编写。 函数 find_robots() 在一个文件名集合上做map操作，并将结果汇总为一个单独的结果， 也就是 find_all_robots() 函数中的 all_robots 集合。 现在，假设你想要修改这个程序让它使用多核CPU。 很简单——只需要**将map()操作替换为一个 concurrent.futures 库中生成的类似操作即可**。

In [52]:
# findrobots.py

import gzip
import io
import glob
from concurrent import futures

def find_robots(filename):
    '''
    Find all of the hosts that access robots.txt in a single log file

    '''
    robots = set()
    with gzip.open(filename) as f:
        for line in io.TextIOWrapper(f,encoding='ascii'):
            fields = line.split()
            if fields[6] == '/robots.txt':
                robots.add(fields[0])
    return robots

def find_all_robots(logdir):
    '''
    Find all hosts across and entire sequence of files
    '''
    files = glob.glob(logdir+'/*.log.gz')
    all_robots = set()
    with futures.ProcessPoolExecutor() as pool:
        for robots in pool.map(find_robots, files):
            all_robots.update(robots)
    return all_robots

if __name__ == '__main__':
    robots = find_all_robots('logs')
    for ipaddr in robots:
        print(ipaddr)